## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Clinical Deidentification Arabic

Deidentification is essential for safeguarding patient privacy in clinical data, including texts, PDFs, images, and DICOM files containing Protected Health Information (PHI). PHI encompasses various health-related data, including common identifiers such as name, address, birth date, and Social Security Number.

- **Model**: `ar.deid.clinical`
- **Model Description**: This pipeline can be used to deidentify Arabic PHI information from medical texts. The PHI information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate CONTACT, NAME, DATE, ID, LOCATION, AGE, PATIENT, HOSPITAL, ORGANIZATION, CITY, STREET, USERNAME, SEX, IDNUM, EMAIL, ZIP, MEDICALRECORD, PROFESSION, PHONE, COUNTRY, DOCTOR, SSN, ACCOUNT, LICENSE, DLN and VIN

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [24]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [25]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [38]:
model_name = "ar-deid-clinical"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [39]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

INFO:sagemaker:Creating model with name: ar-deid-clinical-2024-05-02-18-15-25-696
INFO:sagemaker:Creating endpoint-config with name ar-deid-clinical
INFO:sagemaker:Creating endpoint with name ar-deid-clinical


---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [40]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'


    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
    
    input_data = json.dumps(data, indent=4, ensure_ascii=False) if content_type == 'application/json' else data


    if content_type == 'application/json':
        with open(input_file_name, "w", encoding='utf-8') as f:
            f.write(input_data)
    else:
        with open(input_file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(input_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=data)

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4, ensure_ascii=False)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    with open(output_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_output_dir}/{os.path.basename(output_file_name)}", Body=data)

    return response_data

### Initial Setup

In [41]:
docs = ["""ملاحظات سريرية - مريض السكري:
التاريخ: 30 مايو 2023
اسم المريض: فاطمة علي
العنوان: شارع الحرية، مبنى رقم 456، حي السلام، القاهرة
الرمز البريدي: 67890
البلد: مصر
اسم المستشفى: مستشفى الشفاء
اسم الطبيب: د. محمد صلاح
تفاصيل الحالة:
المريضة فاطمة علي، البالغة من العمر 42 عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية.
الخطة:
تم وصف دواء فموي لخفض مستوى السكر في الدم.
يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام.
يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات.
يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات.
تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.""",

"""ملاحظات سريرية - مريض الربو:
التاريخ: 30 مايو 2023
اسم المريضة: ليلى حسن
العنوان: شارع المعرفة، مبنى رقم 789، حي الأمانة، جدة
الرمز البريدي: 54321
البلد: المملكة العربية السعودية
اسم المستشفى: مستشفى النور
اسم الطبيب: د. أميرة أحمد
تفاصيل الحالة:
المريضة ليلى حسن، البالغة من العمر 35 عامًا، تعاني من مرض الربو المزمن. تشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصها بمرض الربو بناءً على تاريخها الطبي واختبارات وظائف الرئة.
الخطة:
تم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج.
يجب على المريضة حمل معها جهاز الاستنشاق في حالة حدوث نوبة ربو حادة.
يتعين على المريضة تجنب التحسس من العوامل المسببة للربو، مثل الدخان والغبار والحيوانات الأليفة.
يجب مراقبة وظائف الرئة بانتظام ومتابعة التعليمات الطبية المتعلقة بمرض الربو.
تعليم المريضة بشأن كيفية استخدام جهاز الاستنشاق بشكل صحيح وتقنيات التنفس الصحيح""",

"""ملاحظات سريرية - مريض الضغط العالي:
التاريخ: 30 مايو 2023
اسم المريض: خالد محمد
العنوان: شارع الشفاء، مبنى رقم 321، حي السعادة، الإسكندرية
الرمز البريدي: 12345
البلد: مصر
اسم المستشفى: مستشفى الرحمة
اسم الطبيب: د. ليلى علي
تفاصيل الحالة:
المريض خالد محمد، البالغ من العمر 60 عامًا، يعاني من ضغط الدم الارتفاعي. يشكو من الصداع المستمر والدوخة والتعب الشديد. تم تشخيصه بارتفاع ضغط الدم بعد فحص شامل وقياسات متكررة.
الخطة:
تم وصف دواءًا لخفض ضغط الدم ومراقبة المستويات بانتظام.
يجب على المريض مراجعة الطبيب بانتظام لتقييم استجابته للعلاج وضبط الجرعات إن لزم الأمر.
تعليم المريض بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تجنب الأطعمة الغنية بالصوديوم وممارسة الرياضة بانتظام.
يتعين على المريض مراقبة الوزن وتجنب التوتر النفسي والتدخين.
ينصح المريض بإجراء فحوصات الكولسترول والسكر في الدم بانتظام.""",

"""ملاحظات سريرية - مريضة الغدة الدرقية:
التاريخ: 30 مايو 2023
اسم المريضة: سارة عبد الرحمن
العنوان: شارع النجاح، مبنى رقم 987، حي الأمل، القاهرة
الرمز البريدي: 67890
البلد: مصر
اسم المستشفى: مستشفى الأمل
اسم الطبيب: د. أحمد مصطفى
تفاصيل الحالة:
المريضة سارة عبد الرحمن، البالغة من العمر 45 عامًا، تعاني من اضطرابات في الغدة الدرقية. تشكو من زيادة الوزن والتعب والاكتئاب. تم تشخيصها بفرط نشاط الغدة الدرقية بعد فحص الدم والتحاليل المخبرية.
الخطة:
تم وصف العلاج الدوائي لتنظيم وظيفة الغدة الدرقية ومراقبة المستويات الهرمونية بانتظام.
يجب على المريضة مراجعة الطبيب بانتظام لتقييم استجابتها للعلاج وضبط الجرعات إن لزم الأمر.
تعليم المريضة بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تناول الغذاء الصحي وممارسة النشاط البدني بانتظام.
ينصح المريضة بإجراء فحوصات الدم الدورية لمتابعة مستويات الهرمونات وضبط العلاج.
تعليم المريضة بشأن الأعراض التي يجب عليها مراقبتها والتقارير الفورية للطبيب، مثل التعب الشديد أو زيادة الوزن المفرط.""",
]


sample_text = """ملاحظات سريرية - مريض الربو:
التاريخ: 30 مايو 2023
اسم المريض: أحمد سليمان
العنوان: شارع السلام، مبنى رقم 555، حي الصفاء، الرياض
الرمز البريدي: 54321
البلد: المملكة العربية السعودية
اسم المستشفى: مستشفى الأمانة
اسم الطبيب: د. ريم الحمد
تفاصيل الحالة:
المريض أحمد سليمان، البالغ من العمر 30 عامًا، يعاني من مرض الربو المزمن. يشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصه بمرض الربو بناءً على تاريخه الطبي واختبارات وظائف الرئة.
الخطة:
تم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج.
يجب على المريض مراجعة الطبيب بانتظام وتنظيم استخدام جهاز الاستنشاق وفحص وظائف الرئة بانتظام.
ينصح المريض بتجنب التحسس من العوامل المسببة للربو وتجنب التدخين.
يجب على المريض مراقبة الأعراض وتسجيلها والتقارير الفورية للطبيب في حالة حدوث نوبة ربو حادة.
تعليم المريض بشأن تقنيات التنفس الصحيحة وكيفية استخدام جهاز الاستنشاق بشكل صحيح."""

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    - **masked**: Default policy that masks entities with their type.

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (\*).

    - **masked_with_chars**: Masks entities with asterisks (\*).


- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}
```

### JSON


1. **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

2. **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

#### Example 1: masked (default-policy)

In [42]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,ملاحظات سريرية - مريض الربو:\nالتاريخ: [تاريخ] [تاريخ] اسم المريض: [دكتور] العنوان: شارع السلام، مبنى رقم [الرمز البريدي] حي [الموقع] الرمز البريدي: [الرمز البريدي] البلد: [المدينة] [البلد] اسم المستشفى: مستشفى الأمانة اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريض أحمد سليمان، البالغ من العمر [العمر]عامًا، يعاني من مرض الربو المزمن. يشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصه بمرض الربو بناءً على تاريخه الطبي واختبارات وظائف الرئة. الخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج. يجب على المريض مراجعة الطبيب بانتظام وتنظيم استخدام جهاز الاستنشاق وفحص وظائف الرئة بانتظام. ينصح المريض بتجنب التحسس من العوامل المسببة للربو وتجنب التدخين. يجب على المريض مراقبة الأعراض وتسجيلها والتقارير الفورية للطبيب في حالة حدوث نوبة ربو حادة. تعليم المريض بشأن تقنيات التنفس الصحيحة وكيفية استخدام جهاز الاستنشاق بشكل صحيح.


#### Example 2: obfuscated

In [43]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,ملاحظات سريرية - مريض الربو:\nالتاريخ: 30 يونيو 2024 اسم المريض: خضر شحادة العنوان: شارع السلام، مبنى رقم 161، حي شارع المستقبل الرمز البريدي: 09604 البلد: زغوان الغربية أوزبكستان اسم المستشفى: مستشفى الأمانة اسم الطبيب: د. صالحة قاسم تفاصيل الحالة:\nالمريض أحمد سليمان، البالغ من العمر 38عامًا، يعاني من مرض الربو المزمن. يشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصه بمرض الربو بناءً على تاريخه الطبي واختبارات وظائف الرئة. الخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج. يجب على المريض مراجعة الطبيب بانتظام وتنظيم استخدام جهاز الاستنشاق وفحص وظائف الرئة بانتظام. ينصح المريض بتجنب التحسس من العوامل المسببة للربو وتجنب التدخين. يجب على المريض مراقبة الأعراض وتسجيلها والتقارير الفورية للطبيب في حالة حدوث نوبة ربو حادة. تعليم المريض بشأن تقنيات التنفس الصحيحة وكيفية استخدام جهاز الاستنشاق بشكل صحيح.


#### Example 3: masked (default-policy)

In [44]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,ملاحظات سريرية - مريض السكري:\nالتاريخ: [تاريخ] [تاريخ] اسم المريض: [الاسم] العنوان: شارع الحرية، مبنى رقم [الرمز البريدي] حي السلام، [المدينة] الرمز البريدي: [الرمز البريدي] البلد: [البلد] اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر [العمر]عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.
1,ملاحظات سريرية - مريض الربو:\nالتاريخ: [تاريخ] [تاريخ] اسم المريضة: [المريض] العنوان: شارع المعرفة، مبنى رقم [الرمز البريدي] حي [الموقع] الرمز البريدي: [الرمز البريدي] البلد: [المدينة] [البلد] اسم المستشفى: [الموقع] اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريضة [المريض] حسن، البالغة من العمر [العمر]عامًا، تعاني من مرض الربو المزمن. تشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصها بمرض الربو بناءً على تاريخها الطبي واختبارات وظائف الرئة. الخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج. يجب على المريضة حمل معها جهاز الاستنشاق في حالة حدوث نوبة ربو حادة. يتعين على المريضة تجنب التحسس من العوامل المسببة للربو، مثل الدخان والغبار والحيوانات الأليفة. يجب مراقبة وظائف الرئة بانتظام ومتابعة التعليمات الطبية المتعلقة بمرض الربو. \nتعليم المريضة بشأن كيفية استخدام جهاز الاستنشاق بشكل صحيح وتقنيات التنفس الصحيح
2,ملاحظات سريرية - مريض الضغط العالي:\nالتاريخ: [تاريخ] [تاريخ] اسم المريض: [المريض] العنوان: شارع الشفاء، مبنى رقم [٭٭٭٭٭٭٭] حي السعادة، [المدينة] الرمز البريدي: [الرمز البريدي] البلد: [البلد] اسم المستشفى: مستشفى الرحمة اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريض خالد محمد، البالغ من العمر [العمر]عامًا، يعاني من ضغط الدم الارتفاعي. يشكو من الصداع المستمر والدوخة والتعب الشديد. تم تشخيصه بارتفاع ضغط الدم بعد فحص شامل وقياسات متكررة. الخطة:\nتم وصف دواءًا لخفض ضغط الدم ومراقبة المستويات بانتظام. يجب على المريض مراجعة الطبيب بانتظام لتقييم استجابته للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريض بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تجنب الأطعمة الغنية بالصوديوم وممارسة الرياضة بانتظام. يتعين على المريض مراقبة الوزن وتجنب التوتر النفسي والتدخين. ينصح المريض بإجراء فحوصات الكولسترول والسكر في الدم بانتظام.
3,ملاحظات سريرية - مريضة الغدة الدرقية:\nالتاريخ: [تاريخ] [تاريخ] اسم المريضة: [دكتور] العنوان: شارع النجاح، مبنى رقم [الرمز البريدي] حي [المدينة] [المدينة] الرمز البريدي: [الرمز البريدي] البلد: [البلد] اسم المستشفى: مستشفى الأمل اسم الطبيب: د. [الاسم] تفاصيل الحالة:\nالمريضة [المريض] البالغة من العمر [العمر]عامًا، تعاني من اضطرابات في الغدة الدرقية. تشكو من زيادة الوزن والتعب والاكتئاب. تم تشخيصها بفرط نشاط الغدة الدرقية بعد فحص الدم والتحاليل المخبرية. الخطة:\nتم وصف العلاج الدوائي لتنظيم وظيفة الغدة الدرقية ومراقبة المستويات الهرمونية بانتظام. يجب على المريضة مراجعة الطبيب بانتظام لتقييم استجابتها للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريضة بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تناول الغذاء الصحي وممارسة النشاط البدني بانتظام. ينصح المريضة بإجراء فحوصات الدم الدورية لمتابعة مستويات الهرمونات وضبط العلاج. تعليم المريضة بشأن الأعراض التي يجب عليها مراقبتها والتقارير الفورية للطبيب، مثل التعب الشديد أو زيادة الوزن المفرط.


#### Example 4: obfuscated

In [45]:
input_json_data = {"text": docs, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,ملاحظات سريرية - مريض السكري:\nالتاريخ: 30 يونيو 2024 اسم المريض: سهيل مسلم العنوان: شارع الحرية، مبنى رقم 161، حي السلام، القصر الرمز البريدي: 09604 البلد: أوكرانيا اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. نسيمة هندية تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر 58عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.
1,ملاحظات سريرية - مريض الربو:\nالتاريخ: 30 يونيو 2024 اسم المريضة: حمدان فاطمة العنوان: شارع المعرفة، مبنى رقم 540، حي شارع الحيوانات الأليفة الرمز البريدي: 98119 البلد: زغوان الغربية أوزبكستان اسم المستشفى: شارع المدارس اسم الطبيب: د. هاشم جمال تفاصيل الحالة:\nالمريضة شيماء رياضة حسن، البالغة من العمر 26عامًا، تعاني من مرض الربو المزمن. تشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصها بمرض الربو بناءً على تاريخها الطبي واختبارات وظائف الرئة. الخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج. يجب على المريضة حمل معها جهاز الاستنشاق في حالة حدوث نوبة ربو حادة. يتعين على المريضة تجنب التحسس من العوامل المسببة للربو، مثل الدخان والغبار والحيوانات الأليفة. يجب مراقبة وظائف الرئة بانتظام ومتابعة التعليمات الطبية المتعلقة بمرض الربو. \nتعليم المريضة بشأن كيفية استخدام جهاز الاستنشاق بشكل صحيح وتقنيات التنفس الصحيح
2,ملاحظات سريرية - مريض الضغط العالي:\nالتاريخ: 30 يونيو 2024 اسم المريض: شيزوفرين غدير العنوان: شارع الشفاء، مبنى رقم 147، حي السعادة، نابل الجنوبية الرمز البريدي: 82956 البلد: أوكرانيا اسم المستشفى: مستشفى الرحمة اسم الطبيب: د. ندا شناعة تفاصيل الحالة:\nالمريض خالد محمد، البالغ من العمر 67عامًا، يعاني من ضغط الدم الارتفاعي. يشكو من الصداع المستمر والدوخة والتعب الشديد. تم تشخيصه بارتفاع ضغط الدم بعد فحص شامل وقياسات متكررة. الخطة:\nتم وصف دواءًا لخفض ضغط الدم ومراقبة المستويات بانتظام. يجب على المريض مراجعة الطبيب بانتظام لتقييم استجابته للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريض بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تجنب الأطعمة الغنية بالصوديوم وممارسة الرياضة بانتظام. يتعين على المريض مراقبة الوزن وتجنب التوتر النفسي والتدخين. ينصح المريض بإجراء فحوصات الكولسترول والسكر في الدم بانتظام.
3,ملاحظات سريرية - مريضة الغدة الدرقية:\nالتاريخ: 30 يونيو 2024 اسم المريضة: ثابت قدورة العنوان: شارع النجاح، مبنى رقم 213، حي الشبيكة الشمالية القصر الرمز البريدي: 08657 البلد: أوكرانيا اسم المستشفى: مستشفى الأمل اسم الطبيب: د. نورس زينب تفاصيل الحالة:\nالمريضة ثابت قدورة البالغة من العمر 57عامًا، تعاني من اضطرابات في الغدة الدرقية. تشكو من زيادة الوزن والتعب والاكتئاب. تم تشخيصها بفرط نشاط الغدة الدرقية بعد فحص الدم والتحاليل المخبرية. الخطة:\nتم وصف العلاج الدوائي لتنظيم وظيفة الغدة الدرقية ومراقبة المستويات الهرمونية بانتظام. يجب على المريضة مراجعة الطبيب بانتظام لتقييم استجابتها للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريضة بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تناول الغذاء الصحي وممارسة النشاط البدني بانتظام. ينصح المريضة بإجراء فحوصات الدم الدورية لمتابعة مستويات الهرمونات وضبط العلاج. تعليم المريضة بشأن الأعراض التي يجب عليها مراقبتها والتقارير الفورية للطبيب، مثل التعب الشديد أو زيادة الوزن المفرط.


### JSON Lines

1. **Input format**: Provide input in JSON Lines format, where each line is a JSON object representing a text document along with any optional parameters.

```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [46]:
import json

def create_jsonl(records, masking_policy):
    json_records = []

    for text in records:
        record = {
            "text": text,
            "masking_policy": masking_policy
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines


#### Example 1: masked (default-policy)

In [47]:
input_jsonl_data = create_jsonl(docs, masking_policy="masked")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "ملاحظات سريرية - مريض السكري:\nالتاريخ: [تاريخ] [تاريخ] اسم المريض: [الاسم] العنوان: شارع الحرية، مبنى رقم [الرمز البريدي] حي السلام، [المدينة] الرمز البريدي: [الرمز البريدي] البلد: [البلد] اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر [العمر]عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها."}
{"predictions": "ملاحظات سريرية - مريض الربو:\nالتاريخ: [تاريخ] [تاريخ] اسم المريضة: [المريض] العنوان: شارع المعرفة، مبنى رقم [الرمز البريد

#### Example 2: obfuscated

In [48]:
input_jsonl_data = create_jsonl(docs, masking_policy="obfuscated")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "ملاحظات سريرية - مريض السكري:\nالتاريخ: 30 يونيو 2024 اسم المريض: سهيل مسلم العنوان: شارع الحرية، مبنى رقم 161، حي السلام، القصر الرمز البريدي: 09604 البلد: أوكرانيا اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. نسيمة هندية تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر 58عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها."}
{"predictions": "ملاحظات سريرية - مريض الربو:\nالتاريخ: 30 يونيو 2024 اسم المريضة: حمدان فاطمة العنوان: شارع المعرفة، مبنى رقم 161، حي شارع الحيوانات الأليفة الرمز 

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [49]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

INFO:sagemaker:Deleting endpoint with name: ar-deid-clinical
INFO:sagemaker:Deleting endpoint configuration with name: ar-deid-clinical


## 3. Batch inference

In [16]:
import json
import os


input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)


validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"


def write_and_upload_to_s3(input_data, file_name):

    file_format = os.path.splitext(file_name)[1].lower()
    s3_key = f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}"

    if file_format == ".json":
        input_data_json = json.dumps(input_data, indent=4, ensure_ascii=False)

        with open(file_name, "w", encoding='utf-8') as f:
            f.write(input_data_json)
    elif file_format == ".jsonl":
        with open(file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=s3_key, Body=data)

    return input_data


input_json_data = {
    "input1.json": {"text": docs, "masking_policy": "masked"},
    "input2.json": {"text": docs, "masking_policy": "obfuscated"},
    "input3.json": {"text": docs, "masking_policy": "masked_fixed_length_chars"},
    "input4.json": {"text": docs, "masking_policy": "masked_with_chars"}
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}


for file_name, data in input_json_data.items():
    write_and_upload_to_s3(data, f"{json_input_dir}/{file_name}")

for file_name, data in input_jsonl_data.items():
    write_and_upload_to_s3(data, f"{jsonl_input_dir}/{file_name}")



### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [18]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):
    
    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

In [19]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_json_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked


,predictions
0,ملاحظات سريرية - مريض السكري:\nالتاريخ: [تاريخ] [تاريخ] اسم المريض: [الاسم] العنوان: شارع الحرية، مبنى رقم [الرمز البريدي] حي السلام، [المدينة] الرمز البريدي: [الرمز البريدي] البلد: [البلد] اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر [العمر]عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.
1,ملاحظات سريرية - مريض الربو:\nالتاريخ: [تاريخ] [تاريخ] اسم المريضة: [المريض] العنوان: شارع المعرفة، مبنى رقم [الرمز البريدي] حي [الموقع] الرمز البريدي: [الرمز البريدي] البلد: [المدينة] [البلد] اسم المستشفى: [الموقع] اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريضة [المريض] حسن، البالغة من العمر [العمر]عامًا، تعاني من مرض الربو المزمن. تشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصها بمرض الربو بناءً على تاريخها الطبي واختبارات وظائف الرئة. الخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج. يجب على المريضة حمل معها جهاز الاستنشاق في حالة حدوث نوبة ربو حادة. يتعين على المريضة تجنب التحسس من العوامل المسببة للربو، مثل الدخان والغبار والحيوانات الأليفة. يجب مراقبة وظائف الرئة بانتظام ومتابعة التعليمات الطبية المتعلقة بمرض الربو. \nتعليم المريضة بشأن كيفية استخدام جهاز الاستنشاق بشكل صحيح وتقنيات التنفس الصحيح
2,ملاحظات سريرية - مريض الضغط العالي:\nالتاريخ: [تاريخ] [تاريخ] اسم المريض: [المريض] العنوان: شارع الشفاء، مبنى رقم [٭٭٭٭٭٭٭] حي السعادة، [المدينة] الرمز البريدي: [الرمز البريدي] البلد: [البلد] اسم المستشفى: مستشفى الرحمة اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريض خالد محمد، البالغ من العمر [العمر]عامًا، يعاني من ضغط الدم الارتفاعي. يشكو من الصداع المستمر والدوخة والتعب الشديد. تم تشخيصه بارتفاع ضغط الدم بعد فحص شامل وقياسات متكررة. الخطة:\nتم وصف دواءًا لخفض ضغط الدم ومراقبة المستويات بانتظام. يجب على المريض مراجعة الطبيب بانتظام لتقييم استجابته للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريض بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تجنب الأطعمة الغنية بالصوديوم وممارسة الرياضة بانتظام. يتعين على المريض مراقبة الوزن وتجنب التوتر النفسي والتدخين. ينصح المريض بإجراء فحوصات الكولسترول والسكر في الدم بانتظام.
3,ملاحظات سريرية - مريضة الغدة الدرقية:\nالتاريخ: [تاريخ] [تاريخ] اسم المريضة: [دكتور] العنوان: شارع النجاح، مبنى رقم [الرمز البريدي] حي [المدينة] [المدينة] الرمز البريدي: [الرمز البريدي] البلد: [البلد] اسم المستشفى: مستشفى الأمل اسم الطبيب: د. [الاسم] تفاصيل الحالة:\nالمريضة [المريض] البالغة من العمر [العمر]عامًا، تعاني من اضطرابات في الغدة الدرقية. تشكو من زيادة الوزن والتعب والاكتئاب. تم تشخيصها بفرط نشاط الغدة الدرقية بعد فحص الدم والتحاليل المخبرية. الخطة:\nتم وصف العلاج الدوائي لتنظيم وظيفة الغدة الدرقية ومراقبة المستويات الهرمونية بانتظام. يجب على المريضة مراجعة الطبيب بانتظام لتقييم استجابتها للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريضة بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تناول الغذاء الصحي وممارسة النشاط البدني بانتظام. ينصح المريضة بإجراء فحوصات الدم الدورية لمتابعة مستويات الهرمونات وضبط العلاج. تعليم المريضة بشأن الأعراض التي يجب عليها مراقبتها والتقارير الفورية للطبيب، مثل التعب الشديد أو زيادة الوزن المفرط.




Masking policy: obfuscated


,predictions
0,ملاحظات سريرية - مريض السكري:\nالتاريخ: 30 يونيو 2024 اسم المريض: سهيل مسلم العنوان: شارع الحرية، مبنى رقم 161، حي السلام، القصر الرمز البريدي: 09604 البلد: أوكرانيا اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. نسيمة هندية تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر 58عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.
1,ملاحظات سريرية - مريض الربو:\nالتاريخ: 30 يونيو 2024 اسم المريضة: حمدان فاطمة العنوان: شارع المعرفة، مبنى رقم 161، حي شارع الحيوانات الأليفة الرمز البريدي: 09604 البلد: زغوان الغربية أوزبكستان اسم المستشفى: شارع المدارس اسم الطبيب: د. هاشم جمال تفاصيل الحالة:\nالمريضة شيماء رياضة حسن، البالغة من العمر 26عامًا، تعاني من مرض الربو المزمن. تشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصها بمرض الربو بناءً على تاريخها الطبي واختبارات وظائف الرئة. الخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج. يجب على المريضة حمل معها جهاز الاستنشاق في حالة حدوث نوبة ربو حادة. يتعين على المريضة تجنب التحسس من العوامل المسببة للربو، مثل الدخان والغبار والحيوانات الأليفة. يجب مراقبة وظائف الرئة بانتظام ومتابعة التعليمات الطبية المتعلقة بمرض الربو. \nتعليم المريضة بشأن كيفية استخدام جهاز الاستنشاق بشكل صحيح وتقنيات التنفس الصحيح
2,ملاحظات سريرية - مريض الضغط العالي:\nالتاريخ: 30 يونيو 2024 اسم المريض: شيزوفرين غدير العنوان: شارع الشفاء، مبنى رقم 161، حي السعادة، نابل الجنوبية الرمز البريدي: 09604 البلد: أوكرانيا اسم المستشفى: مستشفى الرحمة اسم الطبيب: د. ندا شناعة تفاصيل الحالة:\nالمريض خالد محمد، البالغ من العمر 67عامًا، يعاني من ضغط الدم الارتفاعي. يشكو من الصداع المستمر والدوخة والتعب الشديد. تم تشخيصه بارتفاع ضغط الدم بعد فحص شامل وقياسات متكررة. الخطة:\nتم وصف دواءًا لخفض ضغط الدم ومراقبة المستويات بانتظام. يجب على المريض مراجعة الطبيب بانتظام لتقييم استجابته للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريض بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تجنب الأطعمة الغنية بالصوديوم وممارسة الرياضة بانتظام. يتعين على المريض مراقبة الوزن وتجنب التوتر النفسي والتدخين. ينصح المريض بإجراء فحوصات الكولسترول والسكر في الدم بانتظام.
3,ملاحظات سريرية - مريضة الغدة الدرقية:\nالتاريخ: 30 يونيو 2024 اسم المريضة: ثابت قدورة العنوان: شارع النجاح، مبنى رقم 161، حي الشبيكة الشمالية القصر الرمز البريدي: 09604 البلد: أوكرانيا اسم المستشفى: مستشفى الأمل اسم الطبيب: د. نورس زينب تفاصيل الحالة:\nالمريضة ثابت قدورة البالغة من العمر 57عامًا، تعاني من اضطرابات في الغدة الدرقية. تشكو من زيادة الوزن والتعب والاكتئاب. تم تشخيصها بفرط نشاط الغدة الدرقية بعد فحص الدم والتحاليل المخبرية. الخطة:\nتم وصف العلاج الدوائي لتنظيم وظيفة الغدة الدرقية ومراقبة المستويات الهرمونية بانتظام. يجب على المريضة مراجعة الطبيب بانتظام لتقييم استجابتها للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريضة بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تناول الغذاء الصحي وممارسة النشاط البدني بانتظام. ينصح المريضة بإجراء فحوصات الدم الدورية لمتابعة مستويات الهرمونات وضبط العلاج. تعليم المريضة بشأن الأعراض التي يجب عليها مراقبتها والتقارير الفورية للطبيب، مثل التعب الشديد أو زيادة الوزن المفرط.




Masking policy: masked_fixed_length_chars


,predictions
0,ملاحظات سريرية - مريض السكري:\nالتاريخ: ٭٭٭٭ ٭٭٭٭ اسم المريض: ٭٭٭٭ العنوان: شارع الحرية، مبنى رقم ٭٭٭٭ حي السلام، ٭٭٭٭ الرمز البريدي: ٭٭٭٭ البلد: ٭٭٭٭ اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. ٭٭٭٭ تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر ٭٭٭٭عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.
1,ملاحظات سريرية - مريض الربو:\nالتاريخ: ٭٭٭٭ ٭٭٭٭ اسم المريضة: ٭٭٭٭ العنوان: شارع المعرفة، مبنى رقم ٭٭٭٭ حي ٭٭٭٭ الرمز البريدي: ٭٭٭٭ البلد: ٭٭٭٭ ٭٭٭٭ اسم المستشفى: ٭٭٭٭ اسم الطبيب: د. ٭٭٭٭ تفاصيل الحالة:\nالمريضة ٭٭٭٭ حسن، البالغة من العمر ٭٭٭٭عامًا، تعاني من مرض الربو المزمن. تشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصها بمرض الربو بناءً على تاريخها الطبي واختبارات وظائف الرئة. الخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج. يجب على المريضة حمل معها جهاز الاستنشاق في حالة حدوث نوبة ربو حادة. يتعين على المريضة تجنب التحسس من العوامل المسببة للربو، مثل الدخان والغبار والحيوانات الأليفة. يجب مراقبة وظائف الرئة بانتظام ومتابعة التعليمات الطبية المتعلقة بمرض الربو. \nتعليم المريضة بشأن كيفية استخدام جهاز الاستنشاق بشكل صحيح وتقنيات التنفس الصحيح
2,ملاحظات سريرية - مريض الضغط العالي:\nالتاريخ: ٭٭٭٭ ٭٭٭٭ اسم المريض: ٭٭٭٭ العنوان: شارع الشفاء، مبنى رقم ٭٭٭٭ حي السعادة، ٭٭٭٭ الرمز البريدي: ٭٭٭٭ البلد: ٭٭٭٭ اسم المستشفى: مستشفى الرحمة اسم الطبيب: د. ٭٭٭٭ تفاصيل الحالة:\nالمريض خالد محمد، البالغ من العمر ٭٭٭٭عامًا، يعاني من ضغط الدم الارتفاعي. يشكو من الصداع المستمر والدوخة والتعب الشديد. تم تشخيصه بارتفاع ضغط الدم بعد فحص شامل وقياسات متكررة. الخطة:\nتم وصف دواءًا لخفض ضغط الدم ومراقبة المستويات بانتظام. يجب على المريض مراجعة الطبيب بانتظام لتقييم استجابته للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريض بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تجنب الأطعمة الغنية بالصوديوم وممارسة الرياضة بانتظام. يتعين على المريض مراقبة الوزن وتجنب التوتر النفسي والتدخين. ينصح المريض بإجراء فحوصات الكولسترول والسكر في الدم بانتظام.
3,ملاحظات سريرية - مريضة الغدة الدرقية:\nالتاريخ: ٭٭٭٭ ٭٭٭٭ اسم المريضة: ٭٭٭٭ العنوان: شارع النجاح، مبنى رقم ٭٭٭٭ حي ٭٭٭٭ ٭٭٭٭ الرمز البريدي: ٭٭٭٭ البلد: ٭٭٭٭ اسم المستشفى: مستشفى الأمل اسم الطبيب: د. ٭٭٭٭ تفاصيل الحالة:\nالمريضة ٭٭٭٭ البالغة من العمر ٭٭٭٭عامًا، تعاني من اضطرابات في الغدة الدرقية. تشكو من زيادة الوزن والتعب والاكتئاب. تم تشخيصها بفرط نشاط الغدة الدرقية بعد فحص الدم والتحاليل المخبرية. الخطة:\nتم وصف العلاج الدوائي لتنظيم وظيفة الغدة الدرقية ومراقبة المستويات الهرمونية بانتظام. يجب على المريضة مراجعة الطبيب بانتظام لتقييم استجابتها للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريضة بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تناول الغذاء الصحي وممارسة النشاط البدني بانتظام. ينصح المريضة بإجراء فحوصات الدم الدورية لمتابعة مستويات الهرمونات وضبط العلاج. تعليم المريضة بشأن الأعراض التي يجب عليها مراقبتها والتقارير الفورية للطبيب، مثل التعب الشديد أو زيادة الوزن المفرط.




Masking policy: masked_with_chars


,predictions
0,ملاحظات سريرية - مريض السكري:\nالتاريخ: [٭٭٭٭٭] [٭٭] اسم المريض: [٭٭٭٭٭٭٭] العنوان: شارع الحرية، مبنى رقم [٭٭] حي السلام، [٭٭٭٭٭] الرمز البريدي: [٭٭٭] البلد: [٭] اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. [٭٭٭٭٭٭٭] تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر ٭٭عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها.
1,ملاحظات سريرية - مريض الربو:\nالتاريخ: [٭٭٭٭٭] [٭٭] اسم المريضة: [٭٭٭٭٭٭] العنوان: شارع المعرفة، مبنى رقم [٭٭] حي [٭٭٭٭٭٭٭٭٭٭] الرمز البريدي: [٭٭٭] البلد: [٭٭٭٭٭٭٭٭٭٭٭٭٭] [٭٭٭٭٭٭] اسم المستشفى: [٭٭٭٭٭٭٭٭٭٭] اسم الطبيب: د. [٭٭٭٭٭٭٭٭] تفاصيل الحالة:\nالمريضة [٭٭] حسن، البالغة من العمر ٭٭عامًا، تعاني من مرض الربو المزمن. تشكو من ضيق التنفس والسعال المتكرر والشهيق الشديد. تم تشخيصها بمرض الربو بناءً على تاريخها الطبي واختبارات وظائف الرئة. الخطة:\nتم وصف مضادات الالتهاب غير الستيرويدية والموسعات القصبية لتحسين التنفس وتقليل التهيج. يجب على المريضة حمل معها جهاز الاستنشاق في حالة حدوث نوبة ربو حادة. يتعين على المريضة تجنب التحسس من العوامل المسببة للربو، مثل الدخان والغبار والحيوانات الأليفة. يجب مراقبة وظائف الرئة بانتظام ومتابعة التعليمات الطبية المتعلقة بمرض الربو. \nتعليم المريضة بشأن كيفية استخدام جهاز الاستنشاق بشكل صحيح وتقنيات التنفس الصحيح
2,ملاحظات سريرية - مريض الضغط العالي:\nالتاريخ: [٭٭٭٭٭] [٭٭] اسم المريض: [٭٭٭٭٭٭٭] العنوان: شارع الشفاء، مبنى رقم [٭٭] حي السعادة، [٭٭٭٭٭٭٭٭] الرمز البريدي: [٭٭٭] البلد: [٭] اسم المستشفى: مستشفى الرحمة اسم الطبيب: د. [٭٭٭٭٭٭] تفاصيل الحالة:\nالمريض خالد محمد، البالغ من العمر ٭٭عامًا، يعاني من ضغط الدم الارتفاعي. يشكو من الصداع المستمر والدوخة والتعب الشديد. تم تشخيصه بارتفاع ضغط الدم بعد فحص شامل وقياسات متكررة. الخطة:\nتم وصف دواءًا لخفض ضغط الدم ومراقبة المستويات بانتظام. يجب على المريض مراجعة الطبيب بانتظام لتقييم استجابته للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريض بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تجنب الأطعمة الغنية بالصوديوم وممارسة الرياضة بانتظام. يتعين على المريض مراقبة الوزن وتجنب التوتر النفسي والتدخين. ينصح المريض بإجراء فحوصات الكولسترول والسكر في الدم بانتظام.
3,ملاحظات سريرية - مريضة الغدة الدرقية:\nالتاريخ: [٭٭٭٭٭] [٭٭] اسم المريضة: [٭٭٭٭٭٭٭٭٭٭٭٭٭] العنوان: شارع النجاح، مبنى رقم [٭٭] حي [٭٭٭٭] [٭٭٭٭٭] الرمز البريدي: [٭٭٭] البلد: [٭] اسم المستشفى: مستشفى الأمل اسم الطبيب: د. [٭٭٭٭٭٭٭٭] تفاصيل الحالة:\nالمريضة [٭٭٭٭٭٭٭٭٭٭٭٭٭٭] البالغة من العمر ٭٭عامًا، تعاني من اضطرابات في الغدة الدرقية. تشكو من زيادة الوزن والتعب والاكتئاب. تم تشخيصها بفرط نشاط الغدة الدرقية بعد فحص الدم والتحاليل المخبرية. الخطة:\nتم وصف العلاج الدوائي لتنظيم وظيفة الغدة الدرقية ومراقبة المستويات الهرمونية بانتظام. يجب على المريضة مراجعة الطبيب بانتظام لتقييم استجابتها للعلاج وضبط الجرعات إن لزم الأمر. تعليم المريضة بشأن أهمية اتباع نمط حياة صحي، بما في ذلك تناول الغذاء الصحي وممارسة النشاط البدني بانتظام. ينصح المريضة بإجراء فحوصات الدم الدورية لمتابعة مستويات الهرمونات وضبط العلاج. تعليم المريضة بشأن الأعراض التي يجب عليها مراقبتها والتقارير الفورية للطبيب، مثل التعب الشديد أو زيادة الوزن المفرط.


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [21]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [22]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_jsonlines_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked
{"predictions": "ملاحظات سريرية - مريض السكري:\nالتاريخ: [تاريخ] [تاريخ] اسم المريض: [الاسم] العنوان: شارع الحرية، مبنى رقم [الرمز البريدي] حي السلام، [المدينة] الرمز البريدي: [الرمز البريدي] البلد: [البلد] اسم المستشفى: مستشفى الشفاء اسم الطبيب: د. [دكتور] تفاصيل الحالة:\nالمريضة فاطمة علي، البالغة من العمر [العمر]عامًا، مصابة بمرض السكري من النوع 2. تشكو من زيادة في العطش والجوع المفرط والتبول المتكرر. تم تشخيصها بمرض السكري بعد فحص شامل وفحوصات مخبرية. الخطة:\nتم وصف دواء فموي لخفض مستوى السكر في الدم. يجب على المريضة مراجعة الطبيب بانتظام وإجراء اختبارات السكر في الدم بانتظام. يتعين على المريضة اتباع نظام غذائي صحي ومتوازن، يشمل الحد من استهلاك السكريات والنشويات. يجب مراقبة ضغط الدم والكولسترول أيضًا ومراعاة التعليمات الطبية المتعلقة بتلك الحالات. تعليم المريضة بشأن أعراض الارتفاع أو الانخفاض الحاد في مستوى السكر في الدم وكيفية التعامل معها."}
{"predictions": "ملاحظات سريرية - مريض الربو:\nالتاريخ: [تاريخ] [تاريخ] اسم المريضة: [المريض] العنوان: شارع المعرفة،

In [23]:
model.delete_model()

INFO:sagemaker:Deleting model with name: ar-deid-clinical-2024-05-02-17-41-05-080


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

